<a href="https://colab.research.google.com/github/MinGyeongdo/Sesion2_projcet/blob/main/Moviedata_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install bs4
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=136f0af2231219ff5d7b70a61776336005bc8fcdd31a0b9ba62aee4291a3e9e2
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


- 해당 코드는 다음 블로그 : https://developer-ankiwoong.tistory.com/843?category=847288 를 참고하여 작성하였습니다.

- 참고자료
https://warm-uk.tistory.com/43

In [ ]:
import requests
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from fake_useragent import UserAgent

from time import sleep
from random import randint

In [ ]:
# Useragent 생성 및 header 정보 생성
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent' : ua.random}

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


- 구글의 개발자 도구를 사용하여 해당 블럭에 매칭 되는 HTML 코드를 탐색하고, Feature들을 활용한 평점 예측 모델을 만들기 위하여 최신 데이터를 수집하는 과정입니다.

In [ ]:
titles = []
years = []
imdp_ratings = []
meta_scores = []
votes = []
genres=[]
directors=[]
actors=[]
votes=[]
grosses=[]
movie_rates=[]

In [ ]:
"""
&start=251&ref_=adv_nxt -> 다음페이지 규칙
"""
import re

pages = np.arange(1,50001, 250)
for page in pages:
    page = requests.get(
        "https://www.imdb.com/search/title/?title_type=feature&release_date=1980-01-01,2022-06-30&user_rating=1.0,10.0&colors=color&languages=en&count=250&start="
        + str(page)
        + "&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, "html.parser") # html 파싱
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    sleep(randint(2,10))

    for container in movie_div:
    # 제목  
        name = container.h3.a
        if name == None:
          titles.append(np.nan)
        else:
          titles.append(name.text)
   
    # 개봉년도
        year = container.h3.find('span', class_='lister-item-year')
        if year == None:
          years.append(np.nan)
        else:
          years.append(year.text)
  
    # 장르
        genre = container.find('span', class_='genre')
        if genre == None:
          genres.append(np.nan)
        else:
          genres.append(genre.text)
    
    # 메타스코어
        m_score = container.find('span','metascore')
        if m_score == None:
          meta_scores.append(np.nan)
        else:
          m_score = m_score.text
          meta_scores.append(m_score)
    
    # IMDP rates
        imdp = container.find('strong')
        if imdp == None:
          imdp_ratings.append(np.nan)
        else:
          imdp = imdp.text
          imdp_ratings.append(imdp)

    # Director
          director = container.find('h3', class_ ='lister-item-header').findNext('p').findNext('p').findNext('p')
        if director == None:
          directors.append(np.nan)
        else:
          director = director.text.replace('\n','').replace(' ','').replace(':','').replace('|', '')
          p = re.compile('{}(.*){}'.format(re.escape('Director'), re.escape('Stars')))
          director = p.findall(director)
          directors.append(director)

      # Actor
          actor = container.find('h3', class_ ='lister-item-header').findNext('p').findNext('p').findNext('p')
        if actor == None:
          actors.append(np.nan)
        else:
          actor = actor.text.replace('\n','').replace(' ','').replace(':','').replace('|', '')
          p = re.compile('{}(.*){}'.format(re.escape('Stars'), re.escape(',')))
          actor = p.findall(actor)
          actors.append(actor)

     # Vote
          vote = container.find('span', attrs={'name' : 'nv'})
        if vote == None:
          votes.append(np.nan)
        else:
          vote = vote['data-value']
          votes.append(vote)

     # Movie rates
          rate = container.find('span', class_='certificate')
        if rate == None:
          movie_rates.append(np.nan)
        else:
          movie_rates.append(rate.text)
     # Gross     
    for container in movie_div:
         gross = container.find('p', class_='sort-num_votes-visible')

         if gross == None:
           grosses.append(np.nan)
         else:
           gross = gross.text.replace('\n','')
           p = re.compile('{}(.*){}'.format(re.escape('Gross:'), re.escape('')))
           gross = p.findall(gross)
           if gross == '':
             grosses.append(np.nan)
           else:
             grosses.append(gross)

# Data Frame 생성
movie = pd.DataFrame({
    'titles' : titles,
    'years' : years,
    'imdp_ratings' : imdp_ratings,
    'meta_scores' : meta_scores,
    'votes' : votes,
    'genres' : genres,
    'directors' : directors,
    'actors' : actors,
    'movie_rates' : movie_rates,
    'grosses' : grosses,
})
# 알맞은 데이터수량이 들어 갔는지 확인
print("Titles: ", len(titles))
print("Years: ", len(years))
print("IMDP Ratings: ", len(imdp_ratings))
print("Genres: ", len(genres))
print("META스코어: ", len(meta_scores))
print("Directors : ", len(directors))
print("Actors : ", len(actors))
print("Votes : ", len(votes))
print("Grosses : ", len(grosses))
print("Movie rates : ", len(movie_rates))

movie.to_csv('movies.csv')

Titles:  50000
Years:  50000
IMDP Ratings:  50000
Genres:  50000
META스코어:  50000
Directors :  50000
Actors :  50000
Votes :  50000
Grosses :  50000
Movie rates :  50000


- 해당 코드를 간단하게 설명하면, 1~50000까지의 인기도순으로 정렬한 영화정보 데이터를 추출할 Feature의 Data가 담겨있는 HTML코드를 파싱하여 필요한 데이터를 정규식과 soup라이브러리의 기능을 통해 추출하였습니다. 총 소요시간은 30분 소요되었습니다.